# Recommendation system

# ideas to solve the problem

- we want to recommend the produts to the stores. 
- different stores has different/common products on sales.
- we have stores data related information data
- first we take prodcut sales data and using turicreate library we built recommendation system base on popularity and cosine similarity

- In store table, there are stores which doents have product sale data/information
- we recommend products to these stores base on the similaroty of this stores to another stores which has sales data using cosine similarity 

- Example 
- If store A sales a,b,c,d,e products and store B sales b,c,d,e,f products, and if store A is similar to store B then product sales in store A can be recommendaded in store B.
- In this case product a can be recommendaded to store B and product f can be recommendaded to store A.




In [1]:
# import library
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.cross_validation import train_test_split
import sys

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# load the data
sales = pd.read_csv('Sales_data.csv')
store = pd.read_csv('Store_data.csv')

/anaconda3/lib/python3.6/site-packages/_pytest/fixtures.py:844: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  params = attr.ib(convert=attr.converters.optional(tuple))
/anaconda3/lib/python3.6/site-packages/_pytest/fixtures.py:846: DeprecationWarning: The `convert` argument is deprecated in favor of `converter`.  It will be removed after 2019/01.
  ids = attr.ib(default=None, convert=_ensure_immutable_ids)


# EDA

In [4]:
store.head()

,STORE_CD,LAST_90DAYS_DEPL,LAST_180DAYS_DEPL,STORE_AREA_SQ_FT,STORE_WEEKLY_VOLUME,LAST_365DAYS_DEPL,INCOME_FEATURE_1,INCOME_FEATURE2,INCOME_FEATURE3,INCOME_FEATURE4,...,MALE_POP_5,MALE_POP_6,MALE_POP_7,FEMALE_POP_1,FEMALE_POP_2,FEMALE_POP_3,FEMALE_POP_4,FEMALE_POP_5,FEMALE_POP_6,FEMALE_POP_7
0,STORE_A,541.2502,1018.5504,8,20,1715.3688,0.0231,0.0388,0.0526,0.0711,...,0.0494,0.0295,0.0132,0.0273,0.0877,0.0682,0.0538,0.0518,0.0317,0.0181
1,STORE_B,51.7506,438.1680,55,1350,1149.0872,0.0236,0.0440,0.0585,0.0754,...,0.0484,0.0288,0.0137,0.0270,0.0894,0.0685,0.0520,0.0509,0.0318,0.0192
2,STORE_C,207.9166,428.4165,7,90,812.4164,0.0262,0.0474,0.0523,0.0977,...,0.0313,0.0176,0.0080,0.0275,0.0989,0.0751,0.0458,0.0295,0.0169,0.0106
3,STORE_D,23.8919,54.6919,9,350,113.8689,0.0236,0.0440,0.0585,0.0754,...,0.0484,0.0288,0.0137,0.0270,0.0894,0.0685,0.0520,0.0509,0.0318,0.0192
4,STORE_E,NaN,NaN,8,100,471.3765,0.0236,0.0440,0.0585,0.0754,...,0.0484,0.0288,0.0137,0.0270,0.0894,0.0685,0.0520,0.0509,0.0318,0.0192


In [5]:
store.columns

Index(['STORE_CD', 'LAST_90DAYS_DEPL', 'LAST_180DAYS_DEPL', 'STORE_AREA_SQ_FT',
       'STORE_WEEKLY_VOLUME', 'LAST_365DAYS_DEPL', 'INCOME_FEATURE_1',
       'INCOME_FEATURE2', 'INCOME_FEATURE3', 'INCOME_FEATURE4',
       'INCOME_FEATURE5', 'INCOME_FEATURE6', 'INCOME_FEATURE7',
       'INCOME_FEATURE8', 'MALE_POP_1', 'MALE_POP_2', 'MALE_POP_3',
       'MALE_POP_4', 'MALE_POP_5', 'MALE_POP_6', 'MALE_POP_7', 'FEMALE_POP_1',
       'FEMALE_POP_2', 'FEMALE_POP_3', 'FEMALE_POP_4', 'FEMALE_POP_5',
       'FEMALE_POP_6', 'FEMALE_POP_7'],
      dtype='object')

In [6]:
print(sales.shape)
sales = sales.rename(columns={'STORE_CODE': 'STORE_CD'})
sales.head()

(214, 3)


,STORE_CD,PRODUCT_CODE,LAST_365DAYS_DEPL
0,STORE_A,PRODUCT_1,11.00
1,STORE_A,PRODUCT_2,8.25
2,STORE_A,PRODUCT_3,93.00
3,STORE_A,PRODUCT_4,3.00
4,STORE_A,PRODUCT_5,29.00


In [7]:
sales['STORE_CD'].unique()

array(['STORE_A', 'STORE_B', 'STORE_C', 'STORE_I', 'STORE_K', 'STORE_L',
       'STORE_M', 'STORE_N', 'STORE_R', 'STORE_S', 'STORE_T'],
      dtype=object)

In [8]:
store['STORE_CD'].unique()

array(['STORE_A', 'STORE_B', 'STORE_C', 'STORE_D', 'STORE_E', 'STORE_F',
       'STORE_G', 'STORE_H', 'STORE_I', 'STORE_J', 'STORE_K', 'STORE_L',
       'STORE_M', 'STORE_N', 'STORE_O', 'STORE_P', 'STORE_Q', 'STORE_R',
       'STORE_S', 'STORE_T'], dtype=object)

In [9]:
#sales.groupby('STORE_CD').count()
#sales.groupby('STORE_CD').sum()
#df=sales.merge(store,how='left',on='STORE_CD')
#df.head()

# Recommendation system 

- Store M has only one product product_39 which has last 365 days sales is 0, which create problem in normalization (0/0), so we replace it with very small number 

In [10]:
sales.loc[sales['PRODUCT_CODE']=='PRODUCT_39']

,STORE_CD,PRODUCT_CODE,LAST_365DAYS_DEPL
138,STORE_M,PRODUCT_39,0.0


In [11]:
sales1=sales.copy()
sales1.loc[sales1['PRODUCT_CODE']=='PRODUCT_39','LAST_365DAYS_DEPL']=0.001
sales1.loc[sales1['PRODUCT_CODE']=='PRODUCT_39']

,STORE_CD,PRODUCT_CODE,LAST_365DAYS_DEPL
138,STORE_M,PRODUCT_39,0.001


In [12]:
'''

# create dummy values base on condition on columns
# create column which represent 1 if there is sales and 0 if there is no sales for perticular product
def create_dummy(row):
    if row['LAST_365DAYS_DEPL']==0:
        val=0
    else:
        val=1
    return val

## apply function to create dummy column
data_dummy=sales1.copy()
data_dummy['sales_dummy'] = data_dummy.apply(create_dummy, axis=1)
data_dummy.head()

'''

"\n\n# create dummy values base on condition on columns\n# create column which represent 1 if there is sales and 0 if there is no sales for perticular product\ndef create_dummy(row):\n    if row['LAST_365DAYS_DEPL']==0:\n        val=0\n    else:\n        val=1\n    return val\n\n## apply function to create dummy column\ndata_dummy=sales1.copy()\ndata_dummy['sales_dummy'] = data_dummy.apply(create_dummy, axis=1)\ndata_dummy.head()\n\n"

In [13]:
# create pivot table ofr sales data
df_matrix = pd.pivot_table(sales1, values='LAST_365DAYS_DEPL', index='STORE_CD', columns='PRODUCT_CODE')
# replace NAN with 0
df_matrix=df_matrix.fillna(0)
df_matrix.head()


PRODUCT_CODE,PRODUCT_1,PRODUCT_10,PRODUCT_11,PRODUCT_12,PRODUCT_13,PRODUCT_14,PRODUCT_15,PRODUCT_16,PRODUCT_17,PRODUCT_18,...,PRODUCT_37,PRODUCT_38,PRODUCT_39,PRODUCT_4,PRODUCT_40,PRODUCT_5,PRODUCT_6,PRODUCT_7,PRODUCT_8,PRODUCT_9
STORE_CD,,,,,,,,,,,,,,,,,,,,,
STORE_A,11.0,7.0,179.25,0.0,28.0,0.0,136.0,69.0,7.0,53.0,...,0.0,0.0,0.0,3.0,0.0,29.0,7.0,0.0,14.0,6.0
STORE_B,0.0,1.0,0.00,0.0,28.0,1.0,21.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,0.0,1.0
STORE_C,6.0,13.0,0.00,4.0,11.0,2.0,58.0,38.0,1.0,21.0,...,0.0,0.0,0.0,0.0,0.0,9.0,0.0,9.0,8.0,26.0
STORE_I,10.0,11.0,27.75,8.0,46.0,2.0,158.0,42.0,12.0,62.0,...,0.0,0.0,0.0,0.0,0.0,13.0,12.0,31.0,18.0,15.0
STORE_K,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0


In [14]:
# replace NAN with 0
df_matrix=df_matrix.fillna(0)
df_matrix

PRODUCT_CODE,PRODUCT_1,PRODUCT_10,PRODUCT_11,PRODUCT_12,PRODUCT_13,PRODUCT_14,PRODUCT_15,PRODUCT_16,PRODUCT_17,PRODUCT_18,...,PRODUCT_37,PRODUCT_38,PRODUCT_39,PRODUCT_4,PRODUCT_40,PRODUCT_5,PRODUCT_6,PRODUCT_7,PRODUCT_8,PRODUCT_9
STORE_CD,,,,,,,,,,,,,,,,,,,,,
STORE_A,11.0,7.0,179.25,0.0,28.0,0.0,136.0,69.0,7.0,53.0,...,0.0000,0.0000,0.000,3.0,0.0,29.0,7.0,0.0,14.0,6.0
STORE_B,0.0,1.0,0.00,0.0,28.0,1.0,21.0,1.0,2.0,1.0,...,0.0000,0.0000,0.000,0.0,0.0,0.0,1.0,7.0,0.0,1.0
STORE_C,6.0,13.0,0.00,4.0,11.0,2.0,58.0,38.0,1.0,21.0,...,0.0000,0.0000,0.000,0.0,0.0,9.0,0.0,9.0,8.0,26.0
STORE_I,10.0,11.0,27.75,8.0,46.0,2.0,158.0,42.0,12.0,62.0,...,0.0000,0.0000,0.000,0.0,0.0,13.0,12.0,31.0,18.0,15.0
STORE_K,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.000,0.0,0.0,7.0,0.0,0.0,0.0,0.0
STORE_L,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0000,0.0000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
STORE_M,4.0,87.0,574.50,0.0,142.0,0.0,148.0,1.0,14.0,122.0,...,0.5833,22.7492,0.001,0.0,0.0,76.0,0.0,68.0,48.0,70.0
STORE_N,0.0,0.0,0.00,2.0,68.0,0.0,112.0,0.0,5.0,15.0,...,0.0000,0.0000,0.000,0.0,1.0,30.0,0.0,19.0,18.0,0.0
STORE_R,0.0,0.0,0.00,0.0,1.0,0.0,5.0,1.0,1.0,0.0,...,0.0000,0.0000,0.000,0.0,0.0,0.0,0.0,11.0,0.0,0.0


### Normalization of sales values

- normalization happens across the product values ( column wise )
- In this step, we have normalized the the product, from 0–1 (1 represent the maximum value of the product and 0 is the minimum value of the product)

In [15]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
df_matrix_norm

PRODUCT_CODE,PRODUCT_1,PRODUCT_10,PRODUCT_11,PRODUCT_12,PRODUCT_13,PRODUCT_14,PRODUCT_15,PRODUCT_16,PRODUCT_17,PRODUCT_18,...,PRODUCT_37,PRODUCT_38,PRODUCT_39,PRODUCT_4,PRODUCT_40,PRODUCT_5,PRODUCT_6,PRODUCT_7,PRODUCT_8,PRODUCT_9
STORE_CD,,,,,,,,,,,,,,,,,,,,,
STORE_A,1.000000,0.080460,0.312010,0.00,0.197183,0.0,0.860759,1.000000,0.368421,0.434426,...,0.0,0.0,0.0,1.0,0.0,0.381579,0.583333,0.000000,0.291667,0.085714
STORE_B,0.000000,0.011494,0.000000,0.00,0.197183,0.5,0.132911,0.014493,0.105263,0.008197,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.083333,0.102941,0.000000,0.014286
STORE_C,0.545455,0.149425,0.000000,0.50,0.077465,1.0,0.367089,0.550725,0.052632,0.172131,...,0.0,0.0,0.0,0.0,0.0,0.118421,0.000000,0.132353,0.166667,0.371429
STORE_I,0.909091,0.126437,0.048303,1.00,0.323944,1.0,1.000000,0.608696,0.631579,0.508197,...,0.0,0.0,0.0,0.0,0.0,0.171053,1.000000,0.455882,0.375000,0.214286
STORE_K,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.092105,0.000000,0.000000,0.000000,0.000000
STORE_L,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
STORE_M,0.363636,1.000000,1.000000,0.00,1.000000,0.0,0.936709,0.014493,0.736842,1.000000,...,1.0,1.0,1.0,0.0,0.0,1.000000,0.000000,1.000000,1.000000,1.000000
STORE_N,0.000000,0.000000,0.000000,0.25,0.478873,0.0,0.708861,0.000000,0.263158,0.122951,...,0.0,0.0,0.0,0.0,1.0,0.394737,0.000000,0.279412,0.375000,0.000000
STORE_R,0.000000,0.000000,0.000000,0.00,0.007042,0.0,0.031646,0.014493,0.052632,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.161765,0.000000,0.000000


In [16]:
# convert pivot table to orignal dataframe, which will use as input for models
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_365days_sales'] 
data_norm = pd.melt(d, id_vars=['STORE_CD'], value_name='scaled_365days_sales').dropna()
data_norm.head()
## number of row increase because all stores have all product infomation (11*39)

,STORE_CD,PRODUCT_CODE,scaled_365days_sales
0,STORE_A,PRODUCT_1,1.000000
1,STORE_B,PRODUCT_1,0.000000
2,STORE_C,PRODUCT_1,0.545455
3,STORE_I,PRODUCT_1,0.909091
4,STORE_K,PRODUCT_1,0.000000


### Divide data to training and test set
- we also convert pandas data frame to Sframe to use in Turicreate

In [17]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    Args:
        data (pandas.DataFrame)
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [18]:
train_data, test_data = split_data(sales1)  # original 365 days sales values
train_data_norm, test_data_norm = split_data(data_norm)  # normalised 365 days sales values

In [19]:
print(train_data.shape)
print(test_data.shape)
print(train_data_norm.shape)
print(test_data_norm.shape)

(171, 3)
(43, 3)
(343, 3)
(86, 3)


## Recommendation function using popularity, cosine similarity and pearson 

- we use Turicreate library for recommendation modeling
- we create models base on 3 different way, populariry base, cosine similarity and pearson 

In [20]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display):
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [21]:
# define some globle parameters
user_id = 'STORE_CD'
item_id = 'PRODUCT_CODE'
users_to_recommend = list(sales1[user_id].unique())
n_rec = 5 # number of items to recommend
n_display = 50 # to display the first few rows in an output dataset

### Popularity base models

-  When no target column is provided, the popularity is determined by the number of observations involving each item. 
- When a target is provided, popularity is computed using the item's mean target value.
- When the target column contains sales over 365 days, for example, the model computes the mean sales for each item and uses this to rank items for recommendations.


In [22]:
# models with direct sales as a target values
name = 'popularity'
target = 'LAST_365DAYS_DEPL'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 171 observations with 11 users and 39 items.

Data prepared in: 0.004265s

171 observations to process; with 39 unique items.

+----------+--------------+--------------------+------+
| STORE_CD | PRODUCT_CODE |       score        | rank |
+----------+--------------+--------------------+------+
| STORE_A  |  PRODUCT_29  |        39.0        |  1   |
| STORE_A  |  PRODUCT_30  |        24.0        |  2   |
| STORE_A  |  PRODUCT_38  |      22.7492       |  3   |
| STORE_A  |  PRODUCT_36  |       13.875       |  4   |
| STORE_A  |  PRODUCT_31  |      13.33305      |  5   |
| STORE_B  |  PRODUCT_11  |      195.375       |  1   |
| STORE_B  |  PRODUCT_20  |      123.125       |  2   |
| STORE_B  |  PRODUCT_5   |        29.5        |  3   |
| STORE_B  |  PRODUCT_16  | 25.166666666666668 |  4   |
| STORE_B  |  PRODUCT_30  |        24.0        |  5   |
| STORE_C  |  PRODUCT_19  |      464.375       |  1   |
| STORE_C  |  PRODUCT_11  |      195.375       |  2   |
| STORE_C  |  PRODUCT_20  |      123.125       |  3   |
| STORE_C  |  PRODUCT_15  | 82.85714285714286  |  4   |
| STORE_C  |  PRODUCT_26  |        45.5        |

In [23]:
# models with normalised sales as a target values
name = 'popularity'
target = 'scaled_365days_sales'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 343 observations with 11 users and 39 items.

Data prepared in: 0.005414s

343 observations to process; with 39 unique items.

+----------+--------------+---------------------+------+
| STORE_CD | PRODUCT_CODE |        score        | rank |
+----------+--------------+---------------------+------+
| STORE_A  |  PRODUCT_12  | 0.16666666666666666 |  1   |
| STORE_A  |  PRODUCT_32  |        0.125        |  2   |
| STORE_A  |  PRODUCT_31  |  0.1111111111111111 |  3   |
| STORE_A  |  PRODUCT_22  | 0.07619047619047618 |  4   |
| STORE_A  |  PRODUCT_11  | 0.00805047867711053 |  5   |
| STORE_B  |  PRODUCT_23  | 0.33409090909090905 |  1   |
| STORE_B  |  PRODUCT_25  | 0.28485838779956424 |  2   |
| STORE_B  |  PRODUCT_2   |         0.17        |  3   |
| STORE_B  |  PRODUCT_14  | 0.16666666666666666 |  4   |
| STORE_B  |  PRODUCT_8   | 0.15104166666666666 |  5   |
| STORE_C  |  PRODUCT_5   | 0.19736842105263158 |  1   |
| STORE_C  |  PRODUCT_9   | 0.17755102040816326 |  2   |
| STORE_C  |  PRODUCT_14  | 0.16666666666666666 |  3   |
| STORE_C  |  PRODUCT_16  | 0.16521739130434784 |  4   |
| STORE_C  |  PRODUCT_38  | 0.1

### Collaborative filtering 

- cosine similarity/Pearson base : Create a recommender that uses item-item similarities based on users in common.


In [24]:
name = 'cosine'
target = 'LAST_365DAYS_DEPL'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 171 observations with 11 users and 39 items.

Data prepared in: 0.00427s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+--------------------------------+------------+

Setting up lookup tables.

+----------+--------------+--------------------+------+
| STORE_CD | PRODUCT_CODE |       score        | rank |
+----------+--------------+--------------------+------+
| STORE_A  |  PRODUCT_39  | 1865.3319786381721 |  1   |
| STORE_A  |  PRODUCT_34  | 91.20783054113389  |  2   |
| STORE_A  |  PRODUCT_35  | 37.13945673465729  |  3   |
| STORE_A  |  PRODUCT_29  | 34.951668746471405 |  4   |
| STORE_A  |  PRODUCT_32  | 28.448675091266633 |  5   |
| STORE_B  |  PRODUCT_39  | 198.6305210684475  |  1   |
| STORE_B  |  PRODUCT_23  | 11.21723409075486  |  2   |
| STORE_B  |  PRODUCT_34  | 7.184577474468632  |  3   |
| STORE_B  |  PRODUCT_35  | 4.259339925489928  |  4   |
| STORE_B  |  PRODUCT_32  | 3.1722834486710396 |  5   |
| STORE_C  |  PRODUCT_26  | 11.347194933891297 |  1   |
| STORE_C  |  PRODUCT_8   | 8.577633291482925  |  2   |
| STORE_C  |  PRODUCT_11  |  8.40709325671196  |  3   |
| STORE_C  |  PRODUCT_15  | 6.383392396569252  |  4   |
| STORE_C  |  PRODUCT_5   | 5.478106436133385  |

Processing data in one pass using dense lookup tables.

+-------------------------------------+------------------+-----------------+

In [25]:
name = 'pearson'
target = 'LAST_365DAYS_DEPL'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

+-------------------------------------+------------------+-----------------+

| 2.832ms                             | 20.5             | 8               |

| 5.973ms                             | 100              | 39              |

+-------------------------------------+------------------+-----------------+

...

| Elapsed Time (Item Statistics) | % Complete |

+----------+--------------+--------------------+------+
| STORE_CD | PRODUCT_CODE |       score        | rank |
+----------+--------------+--------------------+------+
| STORE_A  |  PRODUCT_29  | 42.69764308929443  |  1   |
| STORE_A  |  PRODUCT_30  | 24.49019595861435  |  2   |
| STORE_A  |  PRODUCT_38  |      22.7492       |  3   |
| STORE_A  |  PRODUCT_36  |  16.3212655210495  |  4   |
| STORE_A  |  PRODUCT_31  | 15.185181013870238 |  5   |
| STORE_B  |  PRODUCT_11  | 176.86838976019308 |  1   |
| STORE_B  |  PRODUCT_20  | 97.04105002629129  |  2   |
| STORE_B  |  PRODUCT_30  | 23.975090914650966 |  3   |
| STORE_B  |  PRODUCT_38  |      22.7492       |  4   |
| STORE_B  |  PRODUCT_16  | 21.34759361074682  |  5   |
| STORE_C  |  PRODUCT_19  | 459.67193821370597 |  1   |
| STORE_C  |  PRODUCT_11  | 192.65022540986539 |  2   |
| STORE_C  |  PRODUCT_20  | 119.13464323878289 |  3   |
| STORE_C  |  PRODUCT_15  | 78.65264439923422  |  4   |
| STORE_C  |  PRODUCT_26  | 41.822162449359894 |

+--------------------------------+------------+

| 10.977ms                       | 100        |

+--------------------------------+------------+

In [26]:
name = 'cosine'
target = 'scaled_365days_sales'
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Setting up lookup tables.

Processing data in one pass using dense lookup tables.

+-------------------------------------+------------------+-----------------+

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

+-------------------------------------+------------------+-----------------+

...

Training model from provided data.

Gathering per-item and per-user statistics.

+----------+--------------+-----------------------+------+
| STORE_CD | PRODUCT_CODE |         score         | rank |
+----------+--------------+-----------------------+------+
| STORE_A  |  PRODUCT_12  |  0.09364078938961029  |  1   |
| STORE_A  |  PRODUCT_11  |  0.08375277556478977  |  2   |
| STORE_A  |  PRODUCT_31  |  0.08375273086130619  |  3   |
| STORE_A  |  PRODUCT_32  |  0.08375273086130619  |  4   |
| STORE_A  |  PRODUCT_22  |  0.049687065184116364 |  5   |
| STORE_B  |  PRODUCT_25  |  0.020045054810387746 |  1   |
| STORE_B  |  PRODUCT_23  |  0.018344898309026445 |  2   |
| STORE_B  |  PRODUCT_37  |  0.014052599668502808 |  3   |
| STORE_B  |  PRODUCT_8   |  0.013395905494689941 |  4   |
| STORE_B  |  PRODUCT_2   |  0.012052710567201887 |  5   |
| STORE_C  |  PRODUCT_16  |  0.07350521683692932  |  1   |
| STORE_C  |  PRODUCT_31  |  0.06677776177724203  |  2   |
| STORE_C  |  PRODUCT_14  |  0.06677776177724203  |  3   |
| STORE_C  |  PRODUCT_32  |  0.06677776177724203  |  4  

+--------------------------------+------------+

| Elapsed Time (Item Statistics) | % Complete |

In [27]:
name = 'pearson'
target = 'scaled_365days_sales'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

+--------------------------------+------------+

| 449us                          | 100        |

+--------------------------------+------------+

Setting up lookup tables.

Processing data in one pass using dense lookup tables.

...

Gathering per-item and per-user statistics.

+--------------------------------+------------+

+----------+--------------+----------------------+------+
| STORE_CD | PRODUCT_CODE |        score         | rank |
+----------+--------------+----------------------+------+
| STORE_A  |  PRODUCT_12  | 0.19331348004440466  |  1   |
| STORE_A  |  PRODUCT_32  | 0.14751209877431395  |  2   |
| STORE_A  |  PRODUCT_31  | 0.13377614609069297  |  3   |
| STORE_A  |  PRODUCT_22  | 0.09046681679430461  |  4   |
| STORE_A  |  PRODUCT_11  |  0.0289463770192431  |  5   |
| STORE_B  |  PRODUCT_23  |  0.2712101993235675  |  1   |
| STORE_B  |  PRODUCT_25  |  0.2127629170304995  |  2   |
| STORE_B  |  PRODUCT_14  | 0.13582867951620195  |  3   |
| STORE_B  |  PRODUCT_2   |  0.1319540806327547  |  4   |
| STORE_B  |  PRODUCT_8   |  0.1105288111028217  |  5   |
| STORE_C  |  PRODUCT_16  | 0.17261227732119352  |  1   |
| STORE_C  |  PRODUCT_14  | 0.17245950301488241  |  2   |
| STORE_C  |  PRODUCT_5   | 0.17033430923495377  |  3   |
| STORE_C  |  PRODUCT_9   | 0.14764905892261843  |  4   |
| STORE_C  |  

| Elapsed Time (Item Statistics) | % Complete |

+--------------------------------+------------+

# Model evaluation

- we consider model with normalised sales values

In [28]:
models_w_counts = [popularity, cos, pear]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on sales Counts', 'Cosine Similarity on sales Counts', 'Pearson Similarity on sales Counts']
names_w_norm = ['Popularity Model on Scaled sales Counts', 'Cosine Similarity on Scaled sales Counts', 'Pearson Similarity on Scaled sales Counts']

| 356us                          | 100        |

+--------------------------------+------------+

Setting up lookup tables.

Processing data in one pass using dense lookup tables.

+-------------------------------------+------------------+-----------------+

...

| 14.293ms                            | 100              | 39              |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.015803s

In [29]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)

PROGRESS: Evaluate model Popularity Model on sales Counts

Precision and recall summary statistics by cutoff
+--------+---------------------+---------------------+
| cutoff |    mean_precision   |     mean_recall     |
+--------+---------------------+---------------------+
|   1    |  0.7272727272727272 | 0.23484848484848483 |
|   2    | 0.49999999999999994 | 0.27651515151515155 |
|   3    |  0.3333333333333333 | 0.27651515151515155 |
|   4    | 0.29545454545454547 | 0.30303030303030304 |
|   5    | 0.29090909090909095 | 0.34469696969696967 |
|   6    |  0.2727272727272727 |  0.4015151515151515 |
|   7    | 0.28571428571428575 |  0.4848484848484848 |
|   8    | 0.26136363636363635 |  0.4962121212121212 |
|   9    | 0.24242424242424243 |  0.5265151515151516 |
|   10   |  0.2272727272727273 |  0.5416666666666666 |
+--------+---------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 89.48924275176967

Per User RMSE (best)
+----------+--------------------+-------+
|

In [30]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled sales Counts

Precision and recall summary statistics by cutoff
+--------+----------------+---------------------+
| cutoff | mean_precision |     mean_recall     |
+--------+----------------+---------------------+
|   1    |      1.0       | 0.21746359700905155 |
|   2    |      1.0       |  0.3440181031090122 |
|   3    |      1.0       |  0.4705726092089728 |
|   4    |      1.0       |  0.5668240850059032 |
|   5    |      1.0       |  0.6630755608028336 |
|   6    |      1.0       |  0.7411452184179458 |
|   7    |      1.0       |  0.8192148760330578 |
|   8    |      1.0       |  0.8842975206611571 |
|   9    |      1.0       |  0.9380165289256198 |
|   10   |      1.0       |  0.9614325068870523 |
+--------+----------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 0.40013999327215954

Per User RMSE (best)
+----------+---------------------+-------+
| STORE_CD |         rmse        | count |
+----------+------

# Final output files

In [31]:
data_norm.head()

,STORE_CD,PRODUCT_CODE,scaled_365days_sales
0,STORE_A,PRODUCT_1,1.000000
1,STORE_B,PRODUCT_1,0.000000
2,STORE_C,PRODUCT_1,0.545455
3,STORE_I,PRODUCT_1,0.909091
4,STORE_K,PRODUCT_1,0.000000


In [32]:
train_data_norm.head()

STORE_CD,PRODUCT_CODE,scaled_365days_sales
STORE_N,PRODUCT_4,0.0
STORE_B,PRODUCT_13,0.19718309859154928
STORE_C,PRODUCT_25,0.13480392156862744
STORE_A,PRODUCT_20,0.3902439024390244
STORE_T,PRODUCT_32,0.0
STORE_C,PRODUCT_6,0.0
STORE_T,PRODUCT_1,0.0
STORE_I,PRODUCT_17,0.631578947368421
STORE_K,PRODUCT_32,0.0
STORE_R,PRODUCT_28,0.0


In [33]:
def split_data1(data):
    train, test = train_test_split(data, test_size = 0.2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [34]:
data_norm_1,data_norm_2=split_data1(data_norm)
data_norm_1

STORE_CD,PRODUCT_CODE,scaled_365days_sales
STORE_K,PRODUCT_2,0.0
STORE_M,PRODUCT_32,0.0
STORE_M,PRODUCT_26,0.6590909090909091
STORE_A,PRODUCT_2,0.55
STORE_I,PRODUCT_18,0.5081967213114754
STORE_C,PRODUCT_4,0.0
STORE_B,PRODUCT_13,0.19718309859154928
STORE_N,PRODUCT_37,0.0
STORE_L,PRODUCT_18,0.0
STORE_B,PRODUCT_38,0.0


In [35]:
final_model = tc.item_similarity_recommender.create(data_norm_1,
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='scaled_365days_sales', similarity_type='cosine')
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Preparing data set.

Data has 343 observations with 11 users and 39 items.

Data prepared in: 0.004733s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+----------+--------------+-----------------------+------+
| STORE_CD | PRODUCT_CODE |         score         | rank |
+----------+--------------+-----------------------+------+
| STORE_A  |  PRODUCT_20  |  0.14223893284797667  |  1   |
| STORE_A  |  PRODUCT_12  |   0.1403818468252818  |  2   |
| STORE_A  |  PRODUCT_32  |   0.132467778523763   |  3   |
| STORE_A  |  PRODUCT_28  |  0.12904014587402343  |  4   |
| STORE_A  |  PRODUCT_29  |  0.12232717672983805  |  5   |
| STORE_B  |  PRODUCT_15  |  0.014795303344726562 |  1   |
| STORE_B  |  PRODUCT_23  |  0.014597838744521141 |  2   |
| STORE_B  |  PRODUCT_10  |  0.011196691542863846 |  3   |
| STORE_B  |  PRODUCT_36  |  0.01085217110812664  |  4   |
| STORE_B  |  PRODUCT_6   |  0.009509321302175522 |  5   |
| STORE_C  |  PRODUCT_26  |  0.10617608037488215  |  1   |
| STORE_C  |  PRODUCT_31  |  0.09187258523086021  |  2   |
| STORE_C  |  PRODUCT_8   |  0.08753966463023219  |  3   |
| STORE_C  |  PRODUCT_13  |  0.08264730511040523  |  4  

| 863us                               | 10               | 3               |

| 1.394ms                             | 100              | 39              |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

In [36]:
df_rec = recom.to_dataframe()
print(df_rec.shape)
df_rec.head()

Generating candidate set for working with new users.

Finished training in 0.004804s

(55, 4)


,STORE_CD,PRODUCT_CODE,score,rank
0,STORE_A,PRODUCT_20,0.142239,1
1,STORE_A,PRODUCT_12,0.140382,2
2,STORE_A,PRODUCT_32,0.132468,3
3,STORE_A,PRODUCT_28,0.129040,4
4,STORE_A,PRODUCT_29,0.122327,5


In [37]:
# function to create list of recomendation for the list of stores 
# function craete output csv file and save it in local folder

def create_output(model, store_to_recommend, n_rec, print_csv=True):
    recomendation = final_model.recommend(users=store_to_recommend, k=n_rec)
    df_rec = recomendation.to_dataframe()
    df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))
    df_output = df_rec[['STORE_CD', 'recommendedProducts']].drop_duplicates() \
        .sort_values('STORE_CD').set_index('STORE_CD')
    if print_csv:
        df_output.to_csv('option1_recommendation.csv')
        print("An output file can be found in local folder")
    return df_output

In [38]:
df_output = create_output(pear_norm, users_to_recommend, n_rec, print_csv=True)
print(df_output.shape)
df_output.head()

An output file can be found in local folder
(11, 1)


,recommendedProducts
STORE_CD,
STORE_A,PRODUCT_20|PRODUCT_12|PRODUCT_32|PRODUCT_28|PR...
STORE_B,PRODUCT_15|PRODUCT_23|PRODUCT_10|PRODUCT_36|PR...
STORE_C,PRODUCT_26|PRODUCT_31|PRODUCT_8|PRODUCT_13|PRO...
STORE_I,PRODUCT_1|PRODUCT_22|PRODUCT_10|PRODUCT_40|PRO...
STORE_K,PRODUCT_15|PRODUCT_27|PRODUCT_17|PRODUCT_31|PR...


In [39]:
list(df_output.loc['STORE_A'])

['PRODUCT_20|PRODUCT_12|PRODUCT_32|PRODUCT_28|PRODUCT_29']

# Store recomendation function

- given name of the stores, show the recomendation of the products

In [40]:
def product_recomendation(store_id):
    if store_id not in df_output.index:
        print('Store not found.')
        return store_id
    return list(df_output.loc[store_id])

In [41]:
product_recomendation('STORE_A')

['PRODUCT_20|PRODUCT_12|PRODUCT_32|PRODUCT_28|PRODUCT_29']

# Predict recommendation for the stores which was not present in sales data

- use similiarity measure to find out which stores are similar to others stores base on the stores data 
- recommend products to the stores 
- can improve this with eature enginering if more time permited 

In [42]:
store1=store.copy()
store1.head()

,STORE_CD,LAST_90DAYS_DEPL,LAST_180DAYS_DEPL,STORE_AREA_SQ_FT,STORE_WEEKLY_VOLUME,LAST_365DAYS_DEPL,INCOME_FEATURE_1,INCOME_FEATURE2,INCOME_FEATURE3,INCOME_FEATURE4,...,MALE_POP_5,MALE_POP_6,MALE_POP_7,FEMALE_POP_1,FEMALE_POP_2,FEMALE_POP_3,FEMALE_POP_4,FEMALE_POP_5,FEMALE_POP_6,FEMALE_POP_7
0,STORE_A,541.2502,1018.5504,8,20,1715.3688,0.0231,0.0388,0.0526,0.0711,...,0.0494,0.0295,0.0132,0.0273,0.0877,0.0682,0.0538,0.0518,0.0317,0.0181
1,STORE_B,51.7506,438.1680,55,1350,1149.0872,0.0236,0.0440,0.0585,0.0754,...,0.0484,0.0288,0.0137,0.0270,0.0894,0.0685,0.0520,0.0509,0.0318,0.0192
2,STORE_C,207.9166,428.4165,7,90,812.4164,0.0262,0.0474,0.0523,0.0977,...,0.0313,0.0176,0.0080,0.0275,0.0989,0.0751,0.0458,0.0295,0.0169,0.0106
3,STORE_D,23.8919,54.6919,9,350,113.8689,0.0236,0.0440,0.0585,0.0754,...,0.0484,0.0288,0.0137,0.0270,0.0894,0.0685,0.0520,0.0509,0.0318,0.0192
4,STORE_E,NaN,NaN,8,100,471.3765,0.0236,0.0440,0.0585,0.0754,...,0.0484,0.0288,0.0137,0.0270,0.0894,0.0685,0.0520,0.0509,0.0318,0.0192


In [43]:
# replace NAN with 100 for 'LAST_90DAYS_DEPL' column and 200 for 'LAST_180DAYS_DEPL'
# one reason to choose this because 'LAST_365DAYS_DEPL' is 400 for all fissings columns
# we can guess first 90 days sales was 100 , and 180 days sales was 200.
store1[['LAST_90DAYS_DEPL']] = store1[['LAST_90DAYS_DEPL']].fillna(value=100)
store1[['LAST_180DAYS_DEPL']] = store1[['LAST_180DAYS_DEPL']].fillna(value=200)

In [44]:
store2=store1.copy()
store2 = store2.set_index('STORE_CD')
store2.head()

,LAST_90DAYS_DEPL,LAST_180DAYS_DEPL,STORE_AREA_SQ_FT,STORE_WEEKLY_VOLUME,LAST_365DAYS_DEPL,INCOME_FEATURE_1,INCOME_FEATURE2,INCOME_FEATURE3,INCOME_FEATURE4,INCOME_FEATURE5,...,MALE_POP_5,MALE_POP_6,MALE_POP_7,FEMALE_POP_1,FEMALE_POP_2,FEMALE_POP_3,FEMALE_POP_4,FEMALE_POP_5,FEMALE_POP_6,FEMALE_POP_7
STORE_CD,,,,,,,,,,,,,,,,,,,,,
STORE_A,541.2502,1018.5504,8,20,1715.3688,0.0231,0.0388,0.0526,0.0711,0.0792,...,0.0494,0.0295,0.0132,0.0273,0.0877,0.0682,0.0538,0.0518,0.0317,0.0181
STORE_B,51.7506,438.1680,55,1350,1149.0872,0.0236,0.0440,0.0585,0.0754,0.0829,...,0.0484,0.0288,0.0137,0.0270,0.0894,0.0685,0.0520,0.0509,0.0318,0.0192
STORE_C,207.9166,428.4165,7,90,812.4164,0.0262,0.0474,0.0523,0.0977,0.1190,...,0.0313,0.0176,0.0080,0.0275,0.0989,0.0751,0.0458,0.0295,0.0169,0.0106
STORE_D,23.8919,54.6919,9,350,113.8689,0.0236,0.0440,0.0585,0.0754,0.0829,...,0.0484,0.0288,0.0137,0.0270,0.0894,0.0685,0.0520,0.0509,0.0318,0.0192
STORE_E,100.0000,200.0000,8,100,471.3765,0.0236,0.0440,0.0585,0.0754,0.0829,...,0.0484,0.0288,0.0137,0.0270,0.0894,0.0685,0.0520,0.0509,0.0318,0.0192


In [45]:
# calculate store similarity matrix using cosine similarity 
from sklearn.metrics.pairwise import cosine_similarity
ID=store2.index
similarity_values = pd.DataFrame(cosine_similarity(store2), index = ID, columns= ID)
similarity_values

STORE_CD,STORE_A,STORE_B,STORE_C,STORE_D,STORE_E,STORE_F,STORE_G,STORE_H,STORE_I,STORE_J,STORE_K,STORE_L,STORE_M,STORE_N,STORE_O,STORE_P,STORE_Q,STORE_R,STORE_S,STORE_T
STORE_CD,,,,,,,,,,,,,,,,,,,,
STORE_A,1.000000,0.654489,0.994244,0.351533,0.972879,0.879197,0.978796,0.890207,0.991139,0.448196,0.921082,0.163064,0.989559,0.987610,0.978832,0.895862,0.394569,0.418111,0.992566,0.895031
STORE_B,0.654489,1.000000,0.725194,0.922760,0.792864,0.933326,0.769948,0.925975,0.748587,0.957750,0.870411,0.830043,0.753023,0.742291,0.769823,0.919501,0.937183,0.935754,0.734816,0.718463
STORE_C,0.994244,0.725194,1.000000,0.432127,0.991815,0.921610,0.994609,0.928642,0.998521,0.526074,0.943823,0.246689,0.998875,0.990532,0.994626,0.935400,0.473093,0.492097,0.998661,0.908086
STORE_D,0.351533,0.922760,0.432127,1.000000,0.515129,0.742432,0.482860,0.735532,0.468401,0.993437,0.676848,0.980188,0.468772,0.479909,0.482751,0.716273,0.998893,0.995531,0.447451,0.459706
STORE_E,0.972879,0.792864,0.991815,0.515129,1.000000,0.956602,0.999306,0.957771,0.991311,0.605455,0.953002,0.335608,0.993902,0.977800,0.999279,0.967037,0.553328,0.566082,0.990151,0.907146
STORE_F,0.879197,0.933326,0.921610,0.742432,0.956602,1.000000,0.945096,0.998236,0.933401,0.810681,0.973776,0.595358,0.935634,0.926371,0.945075,0.999268,0.771868,0.780427,0.925114,0.861888
STORE_G,0.978796,0.769948,0.994609,0.482860,0.999306,0.945096,1.000000,0.946624,0.992545,0.575387,0.944376,0.300299,0.995164,0.978272,0.999984,0.956895,0.521936,0.535148,0.992222,0.906532
STORE_H,0.890207,0.925975,0.928642,0.735532,0.957771,0.998236,0.946624,1.000000,0.941994,0.803752,0.985039,0.587707,0.942897,0.939297,0.946576,0.997872,0.765715,0.777763,0.933783,0.872495
STORE_I,0.991139,0.748587,0.998521,0.468401,0.991311,0.933401,0.992545,0.941994,1.000000,0.559051,0.959386,0.286734,0.999530,0.995459,0.992584,0.945644,0.508579,0.529016,0.998730,0.909744


In [46]:
# function to calculate most similar stores base on cosine similarity
def similar_store(store):
    similarity_values['store_cd'] = list(similarity_values.index)
    similarity_values_1 = pd.melt(similarity_values, id_vars=['store_cd'])
    similarity_values_1 = similarity_values_1.sort_values(by = ['store_cd', 'value'], ascending = [True, False])
    sim_store=similarity_values_1[(similarity_values_1.store_cd == store) & (similarity_values_1.STORE_CD != store)].head(5)
    return list(sim_store.STORE_CD)


In [47]:
similar_store_list=similar_store('STORE_E')
similar_store_list

['STORE_G', 'STORE_O', 'STORE_M', 'STORE_C', 'STORE_I']

In [48]:
# final recomendation of the product for any stores

def final_reco_prod(store):
    if store in list(sales1.STORE_CD.unique()):
        return product_recomendation(store)
    else:
        similar_store_list=similar_store(store)
        for i in similar_store_list:
            if i in list(sales1.STORE_CD.unique()):
                return product_recomendation(i)

In [49]:
final_reco_prod('STORE_A')

['PRODUCT_20|PRODUCT_12|PRODUCT_32|PRODUCT_28|PRODUCT_29']

In [50]:
fin_arr = []
for i in store.STORE_CD.unique():
    res = final_reco_prod(i)
    res = [i] + res
    fin_arr.append(res)
    

In [51]:
fin_df = pd.DataFrame(fin_arr, columns=['Store_CD', 'Product_Recommendation'])

In [52]:
fin_df

,Store_CD,Product_Recommendation
0,STORE_A,PRODUCT_20|PRODUCT_12|PRODUCT_32|PRODUCT_28|PR...
1,STORE_B,PRODUCT_15|PRODUCT_23|PRODUCT_10|PRODUCT_36|PR...
2,STORE_C,PRODUCT_26|PRODUCT_31|PRODUCT_8|PRODUCT_13|PRO...
3,STORE_D,PRODUCT_13|PRODUCT_15|PRODUCT_30|PRODUCT_18|PR...
4,STORE_E,PRODUCT_25|PRODUCT_18|PRODUCT_8|PRODUCT_24|PRO...
5,STORE_F,PRODUCT_15|PRODUCT_27|PRODUCT_17|PRODUCT_31|PR...
6,STORE_G,PRODUCT_25|PRODUCT_18|PRODUCT_8|PRODUCT_24|PRO...
7,STORE_H,PRODUCT_15|PRODUCT_27|PRODUCT_17|PRODUCT_31|PR...
8,STORE_I,PRODUCT_1|PRODUCT_22|PRODUCT_10|PRODUCT_40|PRO...
9,STORE_J,PRODUCT_13|PRODUCT_15|PRODUCT_30|PRODUCT_18|PR...


In [ ]:
# save in file csv file 
# fin_df.to_csv('store_recom.csv', sep='\t')